

#  Сбор данных: грязная работа вашими руками 

Пришло время самостоятельно написать парсер! Мы будем собирать данные [о ценах на книги.](http://books.toscrape.com)

In [60]:
import requests
import bs4
import pandas as pd

Прогуляйтесь на сайт http://books.toscrape.com/ и изучите его структуру.  


# 1. Сбор ссылок на книги

Напишите функцию `get_soup`, которая по ссылке возвращает html-разметку страницы в формате `bs4` 

In [2]:
def get_page_soup(url_link):
    response = requests.get(url_link)
    html = response.text
    return bs4.BeautifulSoup(html, 'lxml')

In [3]:
main_url = 'http://books.toscrape.com/catalogue/'
page_number = 5

soup = get_page_soup(main_url + f'page-{page_number}.html')
soup.find_all('a')[0].text

'Books to Scrape'

Напишите функцию `get_books_links`, которая находит в html-разметке страницы ссылки на странички с отдельными книгами. 

In [17]:
def get_book_links(page_soup) -> list:
    global main_url
    links = []
    books_block = page_soup.find('ol', {'class': 'row'}).find_all('li')
    for block in books_block:
        link = block.article.div.a.get('href')
        links.append(main_url + link)
    return links

С помощью цикла соберите в лист `book_links` первые 200 книг.

In [18]:
book_links = []

for page_number in range(1, 11):
    soup = get_page_soup(main_url + f'page-{page_number}.html')
    book_links += get_book_links(soup)

In [21]:
# проверка, что задание решено корректно
assert len(book_links) == 200

# 2. Сбор информации о книгах 

Напишите несколько небольших функций, которые собирают различные данные об одной книге, необходимые для ответов на вопросы ниже. Информацию о книге собирайте в виде словаря вида 

```
{ 'name': 'Преступление и наказание', 'rating': 1, 'description': 'ужасно депрессивная книга', ... }

```

In [138]:
def get_book_info(book_link):
    html = requests.get(book_link).text
    soup = bs4.BeautifulSoup(html, 'lxml')
    
    name = soup.find('div', {'class': 'col-sm-6 product_main'}).h1.text
    
    for index, string_rate in enumerate(['Zero', 'One', 'Two', 'Three', 'Four', 'Five']):
        if soup.find('p', {'class': "star-rating " + string_rate}):
            rate = index
            break
    
    try:
        description = soup.select("#content_inner > article > p")[0].text
    except IndexError or AttributeError:
        description = ""
    
    price = soup.find('table', {'class': 'table table-striped'}).find_all('tr')[2].td.text[2:]
    
    tax = soup.find('table', {'class': 'table table-striped'}).find_all('tr')[4].td.text[2:]
    
    return dict(name=name, rating=rate, description=description, price_without_tax=price, tax=tax)

In [143]:
book_info = []

for link in book_links:
    info = get_book_info(link)
    book_info.append(info)

Превратим вектор из информации в полноценную таблицу с данными. 

In [144]:
book_info_df = pd.DataFrame(book_info)
print(book_info_df.shape)
book_info_df.head(5) 

(200, 5)


,name,rating,description,price_without_tax,tax
0,A Light in the Attic,3,It's hard to imagine a world without A Light i...,51.77,0.00
1,Tipping the Velvet,1,"""Erotic and absorbing...Written with starling ...",53.74,0.00
2,Soumission,1,"Dans une France assez proche de la nÃ´tre, un ...",50.10,0.00
3,Sharp Objects,1,"WICKED above her hipbone, GIRL across her hear...",47.82,0.00
4,Sapiens: A Brief History of Humankind,1,From a renowned historian comes a groundbreaki...,54.23,0.00


In [147]:
book_info_df.price_without_tax = book_info_df.price_without_tax.astype(float)
book_info_df.tax = book_info_df.tax.astype(float)

In [148]:
book_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               200 non-null    object 
 1   rating             200 non-null    int64  
 2   description        200 non-null    object 
 3   price_without_tax  200 non-null    float64
 4   tax                200 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 7.9+ KB


- У скольких книг отсутствует описание? Положите число, получившееся в результате ваших манипуляций с таблицей, в переменную `ans1`. 

In [153]:
ans1 = 200 

In [154]:
# проверка, что задание решено корректно

- Сколько раз в данных встречается налог, больший нуля?  Положите число, получившееся в результате ваших манипуляций с таблицей, в переменную `ans2`. 

In [161]:
ans2 = len(book_info_df[book_info_df.tax > 0]) 
ans2

0

In [150]:
# проверка, что задание решено корректно

- Сколько раз рейтинг книги составлял пять звезд? Положите число, получившееся в результате ваших манипуляций с таблицей, в переменную `ans3`. 

In [160]:
ans3 = len(book_info_df[book_info_df.rating == 5]) 
ans3

0

In [157]:
# проверка, что задание решено корректно


- Какова средняя цена книг (без учета налога)? Положите число, получившееся в результате ваших манипуляций с таблицей, в переменную `ans4`. 

In [159]:
ans4 = book_info_df.price_without_tax.mean()
ans4

34.79625

In [ ]:
# проверка, что задание решено корректно
